In [45]:
import sys
sys.path.append('./privatizers')
sys.path.append('./dataProviders')
sys.path.append('./adapters')

from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import fetch_covtype
from adapters import generalAdapter as adapter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dp = fetch_covtype()

X = dp.data
Y = dp.target

In [46]:
# ## Size adjustment
# desired_size = 100
# min_test_size = 0.3
# test_size = 0.3

# total_size = len(X)

# if (desired_size < min_test_size * total_size):
#     test_size = 1-(desired_size / total_size);

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

# ## Dataset
# data_set = {
#     'samples': X_train,
#     'labels': y_train
# }

In [ ]:
## Multiple Dataset sizes
desired_sizes = [100, 1000, 10000, 100000]
data_sets = []

min_test_size = 0.2

total_size = len(X)

for desired_size in desired_sizes:
    test_size = 0.2
    if (desired_size < (1-min_test_size) * total_size):
        test_size = 1-(desired_size / total_size);

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

    ## Dataset
    single_data_set = {
        'samples': X_train,
        'labels': y_train
    }
    data_sets.append(single_data_set)

In [47]:
## SVM without privatization
for data_set in data_sets:
    gnb = GaussianNB()
    gnb.fit(data_set['samples'], data_set['labels'])

    y_pred = gnb.predict(X_test)
    print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

    ## With cross validation
    # scores = cross_val_score(gnb, X, Y, cv=5)
    # for v in scores:
    #     print(str(v).replace('.', ','))

0,5194659432065442


In [48]:
## Privatization: Laplace privatizer
from privatizers import laplacePrivatizer

for data_set in data_sets:
    ad = adapter.GeneralAdapter(7, 1)

    dataInput = ad.fromRaw(data_set['samples'])
    dataTarget = ad.fromRaw(data_set['labels'])

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)

    inputSensitivity = privatizer.getSensitivityList(dataInput)
    targetSensitivity = 7

    epsilon = [1.0, 1.5, 2.0, 2.5, 3.0]

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)
    privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)
    for i in epsilon:
        privatizer = laplacePrivatizer.LaplacePrivatizer(i)
        privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)

        privateTargetsFloat = privatizer.privatize(dataTarget, sensitivityList = targetSensitivity)
        privateTargets = ad.toDiscreteValue(privateTargetsFloat)

        gnb = GaussianNB()
        gnb.fit(privateData, privateTargets)

        y_pred = gnb.predict(X_test)
        print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

0,5161487454209932
0,5168700250640372
0,522075632798083
0,517522447461922
0,5060887019032143
0,5182230699314182
0,5279732558459801
0,554421323711681
0,56664864902085
0,5903665271159831
0,5594255240036357
0,5312577464400804
0,5255598094031454
0,4211050899275622
0,4896541989148098
0,39477407937863224
0,3646146059988432
0,004728771311317377
